In [1]:
import numpy as np
import pandas as pd
import os
import segmentation_models as sm
from IPython.display import display
from glob import glob
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

Segmentation Models: using `tf.keras` framework.


In [2]:
class CFG:
    debug=True

    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
    batch_size=4 if debug else 128 
    input_size=(512,512)
    epochs=2 if debug else 30

In [8]:
def preprocess(path):
    file_bytes=tf.io.read_file(path)
    image=tf.io.decode_jpeg(file_bytes,channels=1)
    image=tf.image.resize(image,CFG.input_size)
    image/=255.0
    return image

def decode_paths(filename):
    original_paths=f"{CFG.dataset_dir}train/"+filename # f"{}"ではTensorの型変換が働かない
    annotated_paths=f"../input/cvc_annotated_cv2/"+filename
    original_imgs=preprocess(original_paths)
    annotated_imgs=preprocess(annotated_paths)
    return original_imgs,annotated_imgs

def build_dataset(paths,shuffle=256):
    AUTO=tf.data.experimental.AUTOTUNE
    dset=tf.data.Dataset.from_tensor_slices(paths)
    dset=dset.map(decode_paths,num_parallel_calls=AUTO)
    dset=dset.cache()
    dset=dset.shuffle(shuffle) if shuffle else dset
    dset=dset.batch(CFG.batch_size).prefetch(AUTO)
    return dset

In [10]:
annotated_filenames=os.listdir("../input/cvc_annotated_cv2/")
train_paths,val_paths=train_test_split(annotated_filenames,test_size=0.25)
train_dset=build_dataset(train_paths)
val_dset=build_dataset(val_paths,shuffle=False)

In [5]:
model=sm.Unet(classes=1,activation="sigmoid",encoder_weights=None,input_shape=CFG.input_size+(1,))

In [11]:
ckpt=tf.keras.callbacks.ModelCheckpoint("./models/unet_cvc/ckpt",save_best_only=True,monitor="val_iou_score",mode="max")

model.compile(optimizer="adam",loss=sm.losses.bce_jaccard_loss,metrics=[sm.metrics.iou_score])
model.fit(train_dset,epochs=CFG.epochs,validation_data=val_dset)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (o